In [1]:
#torch cuda
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import torch.nn as nn
from torch.utils.data import DataLoader

In [3]:
# cargar dataset 

import os
import numpy as np
from PIL import Image

def load_dataset(folder_path):

    # Obtener la lista de archivos en la carpeta
    file_list = os.listdir(folder_path)

    # Filtrar los archivos por extensión (por ejemplo, solo archivos JPG)
    jpg_files = [file_name for file_name in file_list if file_name.endswith(".jpg")]

    # Crear una lista vacía para almacenar las imágenes
    images = []

    # Recorrer la lista de archivos JPG
    for file_name in jpg_files:
        # Combinar la ruta de la carpeta con el nombre del archivo
        file_path = os.path.join(folder_path, file_name)

        # Abrir la imagen usando PIL
        image = Image.open(file_path)

        # Convertir la imagen a un arreglo numpy
        image_np = np.array(image)

        # Agregar la imagen al listado de imágenes
        images.append(image_np)

    # Convertir la lista de imágenes a un arreglo numpy
    dataset = np.array(images, dtype=object)

    return dataset

x_train_path = "/kaggle/input/crack50020220509t090436z001/CRACK500/traincrop/traincrop"
y_train_path = "/kaggle/input/crack50020220509t090436z001/CRACK500/valcrop/valcrop"

x_train = load_dataset(x_train_path)
y_train = load_dataset(y_train_path)
# Verificar la forma del arreglo x_train
print(x_train.shape)
print(y_train.shape)

(1896,)
(348,)


In [4]:
#Cambiar las imagenes de tamño

import cv2
import numpy as np

# Especifica el tamaño deseado para las imágenes
target_size = (64, 64)

def img_resize(target_size, dataset):
    # Crea una lista para almacenar las imágenes redimensionadas
    resized_images = []

    # Itera sobre las imágenes en x_train y redimensiona cada una
    for img in dataset:
        resized_img = cv2.resize(img, target_size)
        resized_images.append(resized_img)

    # Convierte la lista de imágenes redimensionadas en un arreglo numpy
    dataset_resized = np.array(resized_images)
    
    return dataset_resized

x_train_resized = img_resize(target_size, x_train)
y_train_resized = img_resize(target_size, y_train)
print(x_train_resized.shape)
print(y_train_resized.shape)

(1896, 64, 64, 3)
(348, 64, 64, 3)


In [5]:
#Rotar las imagenes para aumentar el dataset

import numpy as np
from scipy.ndimage import rotate

def rotate_images(images, angles=[90, 180, 270]):
    # Crear una lista vacía para almacenar las imágenes rotadas
    rotated_images = []
    
    # Iterar sobre todas las imágenes y ángulos de rotación
    for image in images:
        for angle in angles:
            rotated_image = rotate(image, angle, reshape=False)
            rotated_images.append(rotated_image)
    
    # Convertir la lista de imágenes rotadas en un array numpy
    rotated_images = np.array(rotated_images)
    
    return rotated_images

x_train_resized=rotate_images(x_train_resized)

In [6]:
#Vale necesito pasar las imagenes a un  y normalizarlas

X_train_tensor = torch.from_numpy(x_train_resized).float()
X_train_tensor = X_train_tensor / 255  # Esto lo normalizará al rango de 0 a 1

In [7]:
#Arquitectura de la red
#GENERADOR

class Generator(nn.Module):
    def __init__(self, z_dim=100, channels_img=3, features_g=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # Entrada: Tensor de ruido (N, z_dim, 1, 1)
            self._block(z_dim, features_g * 16, 4, 1, 0),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            self._block(features_g * 2, features_g, 4, 2, 1),  # img: 64x64
            nn.ConvTranspose2d(
                features_g, channels_img, kernel_size=4, stride=2, padding=1
            ),  # img: 128x128
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.gen(x)



In [8]:
#Arquitectura de la red 
#Discriminador 

class Discriminator(nn.Module):
    def __init__(self, channels_img=3, features_d=64):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # Entrada: N x channels_img x 128 x 128
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),  # img: 64x64
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d * 2, 4, 2, 1),  # img: 32x32
            self._block(features_d * 2, features_d * 4, 4, 2, 1),  # img: 16x16
            self._block(features_d * 4, features_d * 8, 4, 2, 1),  # img: 8x8
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),  # img: 4x4
            nn.Sigmoid(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.InstanceNorm2d(out_channels, affine=True),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)



In [11]:
#Entrenamiento normal sin entrenar varias veces el generador
"""
def train(dataloader, generator, discriminator, optim_g, optim_d, loss_fn, device, z_dim):
    for real in dataloader:
        real = real.to(device)
        real = real.permute(0,3,1,2)
        noise = torch.randn((real.size(0), z_dim, 1, 1)).to(device)
        fake = generator(noise)

        ### Actualiza el discriminador ###
        disc_real = discriminator(real).view(-1)
        loss_disc_real = loss_fn(disc_real, torch.ones_like(disc_real))
        disc_fake = discriminator(fake.detach()).view(-1)
        loss_disc_fake = loss_fn(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        discriminator.zero_grad()
        loss_disc.backward()
        optim_d.step()

        ### Actualiza el generador ###
        output = discriminator(fake).view(-1)
        loss_gen = loss_fn(output, torch.ones_like(output))
        generator.zero_grad()
        loss_gen.backward()
        optim_g.step()
        print(f"Epoch [{epoch}/{num_epochs}] Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}")
        return generator,discriminator

"""
a=5




In [12]:
def train(dataloader, generator, discriminator, opt_gen, opt_disc, criterion, device, z_dim, n_gen_updates=2):
    generator.train()
    discriminator.train()

    for epoch in range(num_epochs):
        for batch_idx, real in enumerate(dataloader):
            real = real.to(device)
            noise = torch.randn((batch_size, z_dim, 1, 1)).to(device)

            # Entrenar Discriminador: max log(D(x)) + log(1 - D(G(z)))
            discriminator.zero_grad()
            real=real.permute(0,3,1,2)
            real_output = discriminator(real)
            real_loss = criterion(real_output, torch.ones_like(real_output))
            fake = generator(noise)
            fake_output = discriminator(fake.detach())
            fake_loss = criterion(fake_output, torch.zeros_like(fake_output))
            lossD = real_loss + fake_loss
            lossD.backward()
            opt_disc.step()

            # Entrenar Generador: max log(D(G(z)))
            for _ in range(n_gen_updates):
                generator.zero_grad()
                output = discriminator(fake)
                lossG = criterion(output, torch.ones_like(output))
                if _ == n_gen_updates - 1:
                    lossG.backward()  # en la última iteración no necesitamos mantener el grafo
                else:
                    lossG.backward(retain_graph=True)  # mantener el grafo para futuras iteraciones
                opt_gen.step()

        print(f"Epoch [{epoch}/{num_epochs}] Loss D: {lossD.item()}, loss G: {lossG.item()}")
    return generator, discriminator


In [ ]:
#Hiperparametros
z_dim = 100
lr = 0.0002
batch_size = 32
num_epochs=250

#Discriminador
discriminator = Discriminator().to(device)
generator = Generator(z_dim).to(device)

#Optimizador
opt_disc = torch.optim.Adam(discriminator.parameters(), lr=lr)
opt_gen = torch.optim.Adam(generator.parameters(), lr=lr)

#Criterio
criterion = nn.BCELoss()

#Ruido
fixed_noise = torch.randn((batch_size, z_dim, 1, 1)).to(device)

#Dataloader
dataloader = DataLoader(X_train_tensor, batch_size=batch_size, shuffle=True)

#Entrenamineto
generator,discriminator = train(dataloader, generator, discriminator, opt_gen, opt_disc, criterion, device, z_dim, n_gen_updates=3)

In [ ]:
# Mostrar resultados

import matplotlib.pyplot as plt

# Fijar la semilla del generador de números aleatorios
torch.manual_seed(42)

# Número de imágenes para generar
num_images = 10

# Crear ruido aleatorio
z = torch.randn(num_images, z_dim, 1, 1).to(device)

# Poner el generador en modo evaluación y generar imágenes
generator.eval()
with torch.no_grad():
    fake_imgs = generator(z).detach().cpu()

# Crear una figura para el grid de imágenes
fig, axs = plt.subplots(2, 5, figsize=(15, 6))

# Iterar sobre las imágenes generadas
for i in range(num_images):
    fake_img = fake_imgs[i]

    # Eliminar la primera dimensión y mover los canales de color al final
    fake_img = fake_img.squeeze(0).permute(1, 2, 0)
    fake_img = (fake_img + 1)*0.5  # Esto deshace la normalización típica de [-1, 1] usada en las imágenes de entrada a los GANs

    # Mostrar imagen en el grid
    row = i // 5
    col = i % 5
    axs[row, col].imshow(fake_img)
    axs[row, col].axis('off')  # Quita los ejes para una visualización más clara

plt.tight_layout()
plt.show()
